## Etymology

In [1]:
import pandas as pd
import json

In [2]:
# load data, view dataset size
et = pd.read_csv('../data/etymology.csv')
display(et.head())
print(et.shape)
eng = et[(et['lang'] == 'English')]
print(eng.shape)
eng.head()
test = eng.iloc[:eng.shape[0]//4]
test.shape

,term_id,lang,term,reltype,related_term_id,related_lang,related_term,position,group_tag,parent_tag,parent_position
0,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,group_derived_root,NaN,NaN,NaN,0,0IIU6TlkSA2D7dg1PfxGOw,NaN,NaN
1,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,borrowed_from,fhl-sdgwXaWvYD8CgaKXpQ,Latin,thēsaurus,0,NaN,0IIU6TlkSA2D7dg1PfxGOw,0.0
2,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός,0,NaN,0IIU6TlkSA2D7dg1PfxGOw,1.0
3,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,doublet_with,sqiifJX3XIOQ0vZ0trzRaA,English,treasure,0,NaN,NaN,NaN
4,iwDzanYmWxaUKbOlUy2yyQ,Latin,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός,0,NaN,NaN,NaN


(3884337, 11)
(880736, 11)


(220184, 11)

In [ ]:
# test.to_csv('Final\data\\test_english.csv')
# eng.to_csv('Final\data\\english.csv')

In [65]:
# create language link-node data
et['lang_clean'] = et['lang'].str.strip().str.lower()
et['related_lang_clean'] = et['related_lang'].str.strip().str.lower().fillna('')

langs = set(et['lang_clean'])
langs.update(et['related_lang_clean'])
langs.remove('')
print(langs)
# langs.remove('')
langs = list(langs)
nodes = [{'id': i, 'name': langs[i]} for i in range(len(langs))]
nodes[-5:]

{'mandeali', 'selkup', 'laurentian', 'punic', 'ewondo', 'miya', 'kavalan', 'bodo (india)', 'pacoh', 'ontong java', 'prasuni', 'ruching palaung', 'khvarshi', 'giryama', 'narua', 'kudmali', 'cemuhî', 'nagtipunan agta', 'saramaccan', 'ubykh', 'francisco león zoque', 'catuquinaru', 'serrano', 'pecheneg', 'hunzib', 'gcwi', 'choapan zapotec', 'old danish', 'russian', 'san juan colorado mixtec', 'komodo', 'tugen', 'italiot greek', 'cappadocian greek', "s'gaw karen", 'rusyn', 'biloxi', 'batad ifugao', 'proto-balto-slavic', 'asilulu', 'moroccan arabic', 'kiowa', 'hidatsa', 'faliscan', 'nivkh', 'southern thai', 'central sierra miwok', 'nepali', 'welsh', 'proto-na-dene', 'wathaurong', 'carian', 'hre', 'tagish', 'wardaman', 'ket', 'ajië', 'maypure', 'istro-romanian', 'tundra nenets', 'kerek', 'parthian', 'pare', 'old novgorodian', 'sindarin', 'emerillon', 'proto-ossetic', 'hpon', 'kimaragang', 'shina', 'northern ndebele', 'lower chehalis', 'gmw', 'meru', 'totoli', 'ralte', 'semai', 'kituba', 'talo

[{'id': 2979, 'name': 'pattani malay'},
 {'id': 2980, 'name': 'medieval latin'},
 {'id': 2981, 'name': 'proto-dravidian'},
 {'id': 2982, 'name': 'ligurian'},
 {'id': 2983, 'name': 'nzima'}]

In [68]:
grouped = et[(et['related_lang_clean'] != '') & (et['lang_clean'] != et['related_lang_clean'])].groupby(by=['lang_clean','related_lang_clean','reltype'])[['term']].count().reset_index()
display(grouped.head())
link_dict = {}
for i, row in grouped.iterrows():
    key = (row['lang_clean'].strip().lower(), row['related_lang_clean'].strip().lower())
    if key[::-1] in link_dict: # reverse direction is already in the dict
        key = key[::-1]
    if key not in link_dict: # if pair is new to the dictionary
        link_dict[key] = {'source': langs.index(key[0]), 'target': langs.index(key[1])}
    if row['reltype'] not in link_dict[key]:
        link_dict[key][row['reltype']] = 0
    link_dict[key][row['reltype']] += row['term'] # add the current term count to the dictionary

,lang_clean,related_lang_clean,reltype,term
0,'are'are,are'are,compound_of,6
1,'are'are,hawaiian,cognate_of,2
2,'are'are,maori,cognate_of,2
3,'are'are,proto-austronesian,etymologically_related_to,2
4,'are'are,proto-austronesian,inherited_from,7


In [69]:
links = list(link_dict.values())
for link in links:
    total = sum([val for val in link.values() if isinstance(val, int)])
    link['value'] = total
links[:5]

[{'source': 427, 'target': 2977, 'compound_of': 6, 'value': 3410},
 {'source': 427, 'target': 284, 'cognate_of': 2, 'value': 713},
 {'source': 427, 'target': 2277, 'cognate_of': 2, 'value': 2706},
 {'source': 427,
  'target': 1876,
  'etymologically_related_to': 2,
  'inherited_from': 7,
  'value': 2312},
 {'source': 427, 'target': 2948, 'derived_from': 2, 'value': 3377}]

In [70]:
data_out = {'nodes':nodes,
            'links':links}
# with open('../data/lang_links.json', "w", encoding="utf-8") as outfile: # even if there's an error, 
#         outfile.write(json.dumps(data_out, indent=4))

## Wordnet - not using anymore

Wordnet documentation: https://www.nltk.org/api/nltk.corpus.reader.wordnet.html#module-nltk.corpus.reader.wordnet

In [2]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eviye\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\eviye\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
counts = set()
wordnet.lemmas('run')[0].attributes()
for lemma in wordnet.lemmas('run'):
    counts.add(lemma.count())
counts

print(wordnet.lemmas('run')[0].synset().definition())
print(wordnet.lemmas('run')[0].derivationally_related_forms()[0].synset().definition())

a score in baseball made by a runner touching all four bases safely
make without a miss
